In [ ]:
# default_exp core

# Core

> Helper functions and utilities.

In [ ]:
#export
import requests, json, datetime, collections, typing, urllib
from pathlib import Path
from fastcore.basics import patch_to

In [ ]:
#export
class DotPathDict(collections.UserDict):
    "Wraps a `dict` to allow simple dot notation search of nested `dict`s"
    
    def __getitem__(self, dot_paths):
        "Allows dot search via subscript"
        for dot_path in dot_paths.split(' OR '):
            data, found, path_parts = self.data, True, []
            for path_part in dot_path.split('.'):
                if not isinstance(data, (dict, DotPathDict)):
                    path_parts = '.'.join(path_parts)
                    raise Exception(f'Expected "{path_parts}" to be a `dict` but found {type(data)} {data}')
                path_parts.append(path_part)
                if not path_part in data:
                    found = False
                    break # try the next dot_path, if we have one
                data = data[path_part]
                if isinstance(data, list) and data:
                    data = data[0] # TODO: is it OK to just pull the 1st item from the list?
            if found:
                return DotPathDict(data) if isinstance(data, dict) else data

In [ ]:
test_resource = DotPathDict({
    'resource': {
        'resourceType': 'TestResource', 
        'meta': {
            'versionId': '1', 
            'source': '#dswfkjei2k3'
        }}})

In [ ]:
assert 'TestResource' == test_resource['resource.resourceType']
assert None is test_resource['resource.id']
assert 'TestResource' == test_resource['resource.id OR resource.resourceType']
test_resource_meta = test_resource['resource.meta']
assert '#dswfkjei2k3' == test_resource_meta['source']

In [ ]:
#export
class FhirClient:
    "Helps to GET FHIR resources"
    def __init__(self, api_base:str, x_api_key:str=None, use_local_cache=True):
        self.api_base = api_base
        self.use_local_cache = use_local_cache
        self.request_headers = {}
        if x_api_key is not None:
            self.request_headers['x-api-key'] = x_api_key
        self.default_params = {}    

In [ ]:
#export
def request_to_cache_file(url, params):
    parsed_url = urllib.parse.urlparse(url)
    cache_folder = Path('data/cache/')/parsed_url.netloc
    cache_index = {}
    if (cache_folder/'index.json').is_file():
        with open(cache_folder/'index.json') as f:
            cache_index = json.load(f)
    cache_key = url if params is None else f'{url}?{urllib.parse.urlencode(params)}'
    if cache_key in cache_index:
        return cache_folder/cache_index[cache_key]
    cache_index[cache_key] = f'''{parsed_url.path.replace('/', '')}-{len(cache_index)}'''
    cache_folder.mkdir(parents=True, exist_ok=True)
    with open(cache_folder/'index.json', 'w') as f:
        json.dump(cache_index, f, indent=2)
    return cache_folder/cache_index[cache_key]

In [ ]:
f = request_to_cache_file('http://ips.health/fhir/Patient', dict(_id='fakeId'))
assert f.name.startswith('fhirPatient-')
assert ('data', 'cache', 'ips.health') == f.parent.parts
assert Path('data/cache/ips.health/index.json').is_file()

In [ ]:
#export
class CachedResponse:
    def __init__(self, cache_file):
        self.status_code = 200
        with open(cache_file) as f:
            self.text = f.read()
    def json(self):
        return json.loads(self.text)

In [ ]:
r = CachedResponse('test/data/Patient_bundle.json')
assert '607d0e75-a3b9-4382-a19e-ffac65b1a521' == r.json()['id']
assert 200 == r.status_code

In [ ]:
#export
def requests_get(url, params, headers=None, use_local_cache=True):
    if use_local_cache:
        cache_file = request_to_cache_file(url, params)
        if cache_file.is_file():
            print('GET', cache_file, 'from local cache')
            return CachedResponse(cache_file)
    # HTTP GET from the real FHIR server
    response = requests.get(url, params, headers=headers)
    print('GET', response.url, 'Status', response.status_code)
    # cache the result returned by the real FHIR server
    if use_local_cache and response.status_code == 200:
        cache_file.parent.mkdir(parents=True, exist_ok=True)
        with open(cache_file, 'w') as f:
            f.write(response.text)
    return response

In [ ]:
#export
@patch_to(FhirClient)
def get_as_raw_json(self, resource_type:str, params:dict=None) -> dict: # TODO: rename to get and wrap results
    "GET FHIR resources of `resource_type` in JSON format"
    url = f'{self.api_base}/{resource_type}'
    params = self.default_params if params is None else params
    return requests_get(url, params, headers=self.request_headers).json()

In [ ]:
client = FhirClient('https://ips.health/fhir')
response = client.get_as_raw_json('Patient', dict(_id='fakeId'))
assert isinstance(response, dict)

GET data\cache\ips.health\fhirPatient-1 from local cache


In [ ]:
#export
@patch_to(FhirClient)
def get_next_as_raw_json(self, json_response:dict) -> dict:
    "GET the next set of results"
    for link in json_response['link']:
        if link['relation'] == 'next':
            url = link['url']
            response = requests_get(url, None, self.request_headers, self.use_local_cache)
            print('GET', url, 'Status', response.status_code)
            return response.json()

@patch_to(FhirClient)
def get_all_entries(self, resource_type:str, params:dict=None, page_limit:int=100) -> typing.List[DotPathDict]:
    "Return a list of entries of `resource_type` in JSON format while taking care of bundle pageing"
    page_count, result = 0, []
    bundle = self.get_as_raw_json(resource_type, params)
    total = bundle.get('total', 'Unknown')
    if total == 0:
        print('Returning', len(result), 'entries')
        return result
    while bundle is not None:
        if bundle.get('resourceType', None) != 'Bundle':
            raise Exception(f'Expected a bundle but found', bundle) # might be {'resourceType': 'OperationOutcome' ... 
        result.extend(bundle['entry']) # todo check for OperationOutcome etc in `entry`
        page_count += 1
        if page_count > page_limit:
            print('Stopping early. Will return', len(result), 'entries out of total', total)
            break
        bundle = client.get_next_as_raw_json(bundle)
    def _expected_resource_type(resource):
        actual_resource_type = resource.get('resource', {}).get('resourceType', None)
        if actual_resource_type != resource_type:
            print('Removing resource. Expected', resource_type, 'but found', actual_resource_type)
            return False
        return True
    result = [r for r in result if _expected_resource_type(r)]
    result = [DotPathDict(r) for r in result]
    print('Returning', len(result), 'entries')
    return result

@patch_to(FhirClient)
def get_all_resources(self, resource_type:str, params:dict=None, page_limit:int=100):
    "Return a list of resources of `resource_type` in JSON format"
    result = self.get_all_entries(resource_type, params, page_limit)
    result = [r['resource'] for r in result]
    return result

@patch_to(FhirClient)
def get_by_reference(self, reference:str):
    "Return a resource read from a FHIR server by reference, as a list containg a single bundle entry"
    if reference.startswith(self.api_base):
        reference = reference[len(self.api_base):].strip('/')
    if reference.startswith('http'):
        print(f'WARNING: Found reference {reference} that does not start with {api_base}')
        return []
    resource_type, id = reference.split('/')
    single_resource = self.get_as_raw_json(resource_type, id)
    return [dict(fullUrl = f'{self.api_base}/{resource_type}/{id}', resource = single_resource)]

## Now we can run lots of queries (like &darr;) to load up our local cache
```
client = FhirClient('https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io')

patient_resources = client.get_all_resources('Patient', {
    'birthdate': 'le1996-09-01',
    'gender': 'male,female'
})
```
... so that subsequent calls will use cached results

In [ ]:
patient_resources = client.get_all_resources('Patient', {
    'birthdate': 'le1996-09-01',
    'gender': 'male,female'
})

GET https://ips.health/fhir/Patient?birthdate=le1996-09-01&gender=male%2Cfemale Status 200
Returning 0 entries


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script('00_core.ipynb')

Converted 00_core.ipynb.
